In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
import googlemaps
from datetime import datetime

In [ ]:
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [ ]:
class Order:
    def __init__(self, order_id, customer_location, delivery_deadline):
        self.order_id = order_id
        self.customer_location = customer_location
        self.delivery_deadline = delivery_deadline
        self.assigned_rider = None
        self.delivery_time = None


In [ ]:
class Rider:
    def __init__(self, rider_id, location, capacity, base_location):
        self.rider_id = rider_id
        self.location = location
        self.capacity = capacity
        self.current_load = 0
        self.orders = []
        self.base_location = base_location
        self.earnings = 0

In [ ]:
def calculate_distance(origin, destination):
    directions_result = gmaps.directions(origin, destination)
    if directions_result:
        return directions_result[0]['legs'][0]['distance']['value'] / 1000  # distance in km
    return 0

In [ ]:
def calculate_fuel_cost(distance, cost_per_km):
    return distance * cost_per_km

In [ ]:
def calculate_rider_earnings(rider, orders, mg, incentives):
    earnings = mg
    on_time_deliveries = sum(1 for order in orders if order.delivery_time and order.delivery_time <= order.delivery_deadline)
    if on_time_deliveries / len(orders) >= 0.9:
        earnings += incentives['otd']
    if len(orders) > incentives['order_benchmark']:
        earnings += incentives['extra_orders']
    return earnings

In [ ]:
def assign_orders(orders, riders):
    for order in orders:
        available_riders = [r for r in riders if r.current_load < r.capacity]
        if not available_riders:
            continue
        nearest_rider = min(available_riders, key=lambda r: distance.euclidean(r.location, order.customer_location))
        nearest_rider.orders.append(order)
        nearest_rider.current_load += 1
        order.assigned_rider = nearest_rider

In [ ]:
def optimize_routes(orders, riders):
    for rider in riders:
        if rider.orders:
            waypoints = [order.customer_location for order in rider.orders]
            directions_result = gmaps.directions(
                origin=rider.location,
                destination=waypoints[-1],
                waypoints=waypoints[:-1],
                optimize_waypoints=True
            )
            if directions_result:
                rider.route = directions_result
                for order in rider.orders:
                    order.delivery_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            else:
                print("Error: No directions found")

In [ ]:
def track_riders(riders):
    for rider in riders:
        if rider.orders:
            rider.location = rider.orders[0].customer_location


In [ ]:
def calculate_metrics(orders, riders, mg, incentives, cost_per_km):
    total_orders = len(orders)
    on_time_deliveries = sum(1 for order in orders if order.delivery_time and order.delivery_time <= order.delivery_deadline)
    otd_rate = on_time_deliveries / total_orders

    total_cost = 0
    for rider in riders:
        first_mile_distance = calculate_distance(rider.base_location, rider.orders[0].customer_location) if rider.orders else 0
        last_mile_distance = sum(calculate_distance(order.customer_location, rider.orders[i+1].customer_location) for i, order in enumerate(rider.orders[:-1]))
        return_mile_distance = calculate_distance(rider.orders[-1].customer_location, rider.base_location) if rider.orders else 0

        total_distance = first_mile_distance + last_mile_distance + return_mile_distance
        fuel_cost = calculate_fuel_cost(total_distance, cost_per_km)
        rider.earnings = calculate_rider_earnings(rider, rider.orders, mg, incentives)
        total_cost += fuel_cost + rider.earnings

    return otd_rate, total_cost

In [ ]:
def main():
    orders = [Order(1, (28.7041, 77.1025), '2024-08-12 12:00:00'), Order(2, (28.5355, 77.3910), '2024-08-12 13:00:00')]
    riders = [Rider(1, (28.7041, 77.1025), 5, (28.7041, 77.1025)), Rider(2, (28.5355, 77.3910), 5, (28.5355, 77.3910))]

    mg = 500  
    incentives = {
        'otd': 100,  
        'order_benchmark': 5,  
        'extra_orders': 50  
    }
    cost_per_km = 2.5  

    assign_orders(orders, riders)
    optimize_routes(orders, riders)
    track_riders(riders)
    otd_rate, total_cost = calculate_metrics(orders, riders, mg, incentives, cost_per_km)

    print(f"On Time Delivery Rate: {otd_rate * 100:.2f}%")
    print(f"Total Cost: Rs. {total_cost:.2f}")

if __name__ == "__main__":
    main()
